In [1]:
import sys, math, logging
import numpy as np, pandas as pd, geopandas as gpd
import gurobipy as gp
from gurobipy import GRB, quicksum
import networkx as nx
from datetime import datetime
from time import perf_counter

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

In [2]:
# Load spatially smaller-sample data
shapefileDemands = "small/smallHexGrid.shp"
parking = "small/CharParkSmall.shp"
csv_distance_path = "small/shortestSmall.csv"
spotPrices = "ElPrice.csv"

def load_data():
    MONTHS = ['January','February','March','April','May','June','July','August','September','October','November','December']
    M_ABBR = {'January':'Jan','February':'Feb','March':'Mar','April':'Apr','May':'May','June':'Jun','July':'Jul','August':'Aug','September':'Sep','October':'Oct','November':'Nov','December':'Dec'}
    H = list(range(1,49))
    N_MONTH = {'January':31,'February':28,'March':31,'April':30,'May':31,'June':30,'July':31,'August':31,'September':30,'October':31,'November':30,'December':31}
    DAYS = sum(N_MONTH.values())
    SLOT_HOURS = 0.5
    C_pub = ['slow','medium','fast']
    K = {'slow':5.5,'medium':11.0,'fast':25.0}
    Price = {'slow':5.50,'medium':6.00,'fast':6.50}

    raw = pd.read_csv(spotPrices)
    tou = {m:{t: retail(raw.at[(t-1)//2, M_ABBR[m]]) for t in H} for m in MONTHS}
    monthly_kwh_per_kWp = {'January':1.16,'February':2.47,'March':3.87,'April':5.27,'May':6.12,'June':6.42,'July':5.96,'August':5.05,'September':3.77,'October':2.33,'November':1.07,'December':0.67}
    base_hour_cf = np.array([0,0,0,0,0,0.02,0.04,0.06,0.10,0.15,0.20,0.20,0.20,0.16,0.12,0.08,0.04,0,0,0,0,0,0,0])
    base_hh_cf = np.repeat(base_hour_cf, 2)
    E_base = base_hh_cf.sum()*0.5
    pv_cf = {m:{t: min((monthly_kwh_per_kWp[m]/E_base)*base_hh_cf[t-1], 1.0) for t in H} for m in MONTHS}
    PV_KWH_PER_SLOT_AT_CF1 = 0.40*0.5

    gdf = gpd.read_file(shapefileDemands)
    gdf["HexID"] = gdf["HexID"].astype(int)
    gdf["TimeIndex"] = gdf["TimeIndex"].astype(int)
    gdf["charType"] = gdf["charType"].str.lower().str.strip()
    HexIDs = sorted(gdf["HexID"].unique())
    time_indices = sorted(gdf["TimeIndex"].unique())

    parking_gdf = gpd.read_file(parking)
    parking_gdf["HexID"] = parking_gdf["HexID"].astype(int)
    parking_gdf['Panels'] = (parking_gdf['Panels']*0.5).astype(int)
    parking_gdf['ParkingCap'] += 17
    parking_gdf['homeChar'] += 5
    CL = parking_gdf.set_index("HexID")["ParkingCap"].to_dict()
    home_avail = parking_gdf.set_index("HexID")["homeChar"].to_dict()
    pv_upper = parking_gdf.rename(columns={'Panels':'PV_upper'}).set_index("HexID")["PV_upper"].to_dict()
    for i in HexIDs: CL.setdefault(i,0); home_avail.setdefault(i,0); pv_upper.setdefault(i,0)

    # Big-M for redirection coupling: max kWh a site j can physically deliver in a slot
    M_redir = {j: SLOT_HOURS * CL[j] * max(K.values()) for j in HexIDs}

    grp = (gdf[["HexID","TimeIndex","Demand","charType"]].groupby(["HexID","TimeIndex","charType"], as_index=False)["Demand"].sum())
    D_home = {(i,t):0.0 for i in HexIDs for t in time_indices}
    D_work = {(i,t):0.0 for i in HexIDs for t in time_indices}
    D_pub  = {(i,t):0.0 for i in HexIDs for t in time_indices}
    for _, r in grp.iterrows():
        i, t, val = int(r.HexID), int(r.TimeIndex), float(r.Demand)
        if r.charType == "home": D_home[(i,t)] += val
        elif r.charType == "work": D_work[(i,t)] += val
        else: D_pub[(i,t)] += val
    HOME_KWH_PER_SLOT = 3.5
    for i in HexIDs:
        cap_i = home_avail[i] * HOME_KWH_PER_SLOT
        for t in time_indices:
            D_home[(i,t)] = max(D_home[(i,t)] - cap_i, 0.0)

    Demand = {}
    for i in HexIDs:
        for m in MONTHS:
            for t in H:
                Demand[(i,m,t,'home')]   = D_home[(i,t)]
                Demand[(i,m,t,'public')] = D_work[(i,t)] + D_pub[(i,t)]

    dist = get_distance_dict(csv_distance_path)
    MAX_DIST = 1.5
    allowed_pairs = {(i,j) for (i,j), d in dist.items() if i!=j and d<=MAX_DIST}

    def arc_active(i,j,t): return (D_pub[(i,t)] + D_pub[(j,t)]) > 1e-3
    A = {(i,j,m,t) for (i,j) in allowed_pairs for m in MONTHS for t in H if arc_active(i,j,t)}

    speed_car, speed_walk, value_time = 30.0, 6.0, 80.0
    beta = value_time * (1.0/speed_car + 2.0/speed_walk)
    T = {(i,j): beta * dist[(i,j)] for (i,j) in allowed_pairs}
    x_kWh = 20.0
    redir_min = 0.10

    DISC = 0.06
    ASSETS = {'slow':(8500,10,0.00),'medium':(16500,10,0.00),'fast':(220000,10,0.00),'PV':(4500,25,0.00),'Batt':(40000,15,0.00)}
    daily_cost = {g: ASSETS[g][0]*pvf_daily(DISC,ASSETS[g][1]) + ASSETS[g][0]*ASSETS[g][2]/365.0 for g in ASSETS}
    # Small per-port opex to discourage many low-kW stalls
    PER_STALL_OPEX = 0.30  # SEK/day per installed plug (tune)
    daily_cost['slow']   += PER_STALL_OPEX
    daily_cost['medium'] += PER_STALL_OPEX
    daily_cost['fast']   += PER_STALL_OPEX

    RHO = 2
    prev_month = {MONTHS[i]: MONTHS[i-1] if i>0 else MONTHS[-1] for i in range(len(MONTHS))}
    prev_dict = {t:(t-1 if t>1 else 48) for t in H}
    BATT_MAX = 400
    BATT_CELL_CAP = 10
    C_RATE_KW = 11
    BP = 0.8
    M_CH = (C_RATE_KW*0.5)*BP
    penalty_slack = 1000.0

    G = nx.Graph(); G.add_nodes_from(HexIDs); G.add_edges_from(allowed_pairs)
    pub_demand = {i: sum(D_pub[(i,t)] + D_work[(i,t)] for t in time_indices) for i in HexIDs}
    for comp in nx.connected_components(G):
        if (any(pub_demand[i] > 0 for i in comp) and all(CL[i] == 0 for i in comp)):
            i_star = max(comp, key=lambda i: pub_demand[i]); CL[i_star] = 2

    return {
        'I':HexIDs,'MONTHS':MONTHS,'H':H,'N_MONTH':N_MONTH,'DAYS':DAYS,
        'C':C_pub,'K':K,'Price':Price,'tou':tou,
        'Demand':Demand,'CL':CL,'home_avail':home_avail,
        'pv_cf':pv_cf,'PV_slot_at_cf1':PV_KWH_PER_SLOT_AT_CF1,'pv_upper':pv_upper,
        'A':A,'T':T,'x_kWh':x_kWh,'redir_min':redir_min,'M_redir': M_redir,
        'daily_cost':daily_cost,'BATT_MAX':BATT_MAX,'BATT_CELL_CAP':BATT_CELL_CAP,
        'eta_ch':0.95,'eta_dis':0.95,'RHO':RHO,'prev_month':prev_month,'prev_dict':prev_dict,
        'M_CH':M_CH,'penalty_slack':penalty_slack
    }

def retail(p_ore):
    """Convert p_ore (öre/kWh) to SEK/kWh, then add grid‐fee, tax & VAT."""
    p_sek = p_ore / 100.0                   # öre→SEK
    VAT, TAX, GRID, MARKUP = 0.25, 0.375, 0.45, 0.02
    return (p_sek + TAX + GRID + MARKUP) * (1 + VAT)

def get_distance_dict(csv_path):
    df = pd.read_csv(csv_path)
    df["distance_km"] = df["distance"] / 1000
    return {(int(r.from_HexID), int(r.to_HexID)): round(r.distance_km, 4) for _, r in df.iterrows()}

def pvf_daily(r: float, n: int) -> float:
    """Present-value factor: SEK upfront → SEK/day over n years."""
    return (r*(1+r)**n)/((1+r)**n - 1) / 365.0

In [3]:
class Column:
    def __init__(self, i, x, pv, batt, cap_total, fixed_cost):
        self.i = i
        self.x = x
        self.pv = pv
        self.batt = batt
        self.cap_total = cap_total
        self.fixed_cost = fixed_cost
    
def initial_columns(data):
    cols = []
    I, C = data['I'], data['C']
    K, CL = data['K'], data['CL']
    pv_upper = data['pv_upper']; dc = data['daily_cost']
    for i in I: cols.append(Column(i, {c:0 for c in C}, 0, 0, 0.0, 1e-6))
    
    # Seed columns based on demand patterns
    for i in I:
        peak_demand = max(data['Demand'][(i, m, t, 'public')] for m in data['MONTHS'] for t in data['H'])
        charger_combinations = [
            {'slow': min(CL[i], math.ceil(peak_demand / K['slow']))},
            {'medium': min(CL[i], math.ceil(peak_demand / K['medium']))},
            {'fast': min(CL[i], math.ceil(peak_demand / K['fast']))},
            {'slow': min(CL[i]//2, 1), 'medium': min(CL[i]//2, 1)},
            {'medium': min(CL[i]//2, 1), 'fast': min(CL[i]//2, 1)}
        ]
        
        for combo in charger_combinations:
            x = {c: 0 for c in C}
            for c, count in combo.items():
                x[c] = count
            cap = sum(K[c] * x[c] for c in C)
            fix = sum(dc[c] * x[c] for c in C)
            cols.append(Column(i, x, 0, 0, cap, fix))

        # Full-capacity seed (lets LP serve demand immediately if profitable)
        if CL[i] > 0:
            x_full = {c: 0 for c in C}
            # Put all stalls into the most versatile type (medium), as a strong seed
            x_full['medium'] = CL[i]
            cap = sum(K[c] * x_full[c] for c in C)
            fix = sum(dc[c] * x_full[c] for c in C)
            cols.append(Column(i, x_full, 0, 0, cap, fix))

        # Composite columns so λ can pick chargers + PV + battery from the start
        if CL[i] > 0:
            # A few representative mixes anchored on medium chargers
            med_needed = max(1, min(CL[i], math.ceil(peak_demand / K['medium'])))
            pv_choices   = [0, min(int(0.25*pv_upper[i]), pv_upper[i]), min(int(0.5*pv_upper[i]), pv_upper[i]), min(int(0.75*pv_upper[i]), pv_upper[i])]
            batt_choices = [0, 1, 2, 5, 10] 
            mixes = [
                # medium-only baseline
                ({'slow': 0, 'medium': med_needed, 'fast': 0}, 0, 0),
                # medium + modest PV
                ({'slow': 0, 'medium': med_needed, 'fast': 0}, min(5, pv_upper[i]), 0),
                # medium + modest battery
                ({'slow': 0, 'medium': med_needed, 'fast': 0}, 0, 5),
                # medium + PV + battery (seed the arbitrage option)
                ({'slow': 0, 'medium': med_needed, 'fast': 0}, min(5, pv_upper[i]), 5),
                # slightly bigger PV/batt alternative
                ({'slow': 0, 'medium': min(CL[i], med_needed+1), 'fast': 0}, min(10, pv_upper[i]), 10),
            ]
            for xcombo, pv_cnt, batt_cnt in mixes:
                x = {c: 0 for c in C}; x.update(xcombo)
                cap = sum(K[c] * x[c] for c in C)
                fix = sum(dc[c] * x[c] for c in C) + dc['PV']*pv_cnt + dc['Batt']*batt_cnt
                cols.append(Column(i, x, pv_cnt, batt_cnt, cap, fix))

            for pv_cnt in pv_choices:
                for batt_cnt in batt_choices:
                    xcombo = {'slow': 0, 'medium': min(CL[i], med_needed + (1 if pv_cnt or batt_cnt else 0)), 'fast': 0}
                    x = {c: 0 for c in C}; x.update(xcombo)
                    cap = sum(K[c] * x[c] for c in C)
                    fix = sum(dc[c] * x[c] for c in C) + dc['PV']*pv_cnt + dc['Batt']*batt_cnt
                    cols.append(Column(i, x, pv_cnt, batt_cnt, cap, fix))

    return cols

In [4]:
def solve_with_repricing(data, max_outer=5, max_cg=30):
    cols = initial_columns(data)
    prev_duals = None
    prev_lambda_vals = None
    best_obj = -1e100
    convergence_count = 0
    column_pool = {}  # Store columns by site
    
    for outer in range(max_outer):
        # pure CG on LP
        best = -1e100
        for it in range(max_cg):
        # strong penalty in LP master to “price out” slack
            rmp, duals, lam_vals = solve_RMP(cols, data, integrality=False,
                                         slack_penalty_override=1e4,
                                         prev_lambda_vals=prev_lambda_vals,
                                         prox_weight=5e-3)
            if rmp.status != GRB.OPTIMAL: 
                break
            prev_lambda_vals = lam_vals  # feed prox next time
        
            # Store the objective value in a variable
            current_obj_val = rmp.objVal
            logging.info(f"[Outer {outer} | CG {it}] LP-RMP obj = {current_obj_val:,.2f}  | cols={len(cols)}")
            
            # RC diagnostics
            top = []
            for i in data['I']:
                _, rc = solve_pricer_for_site(i, duals, data, prev_duals=prev_duals)
                top.append(rc)
            logging.info(f"  RC max/min/avg: {max(top):.1f} / {min(top):.1f} / {sum(top)/len(top):.1f}")
    
            if current_obj_val <= best + 1e-4 and it > 0:
                break
            best = current_obj_val
    
            #  pricing
            new_cols = []   
            for i in data['I']:
                col, rc = solve_pricer_for_site(i, duals, data, prev_duals=prev_duals)
                if (col is not None) and (rc > 1e-6):
                    # simple de-dup at source
                    if not any((col.i == e.i and col.pv == e.pv and col.batt == e.batt and
                                all(col.x[c] == e.x[c] for c in data['C'])) for e in cols):
                        new_cols.append(col)
    
            if not new_cols:
                logging.info("No positive-reduced-cost columns at this LP.")
                break
    
            cols.extend(new_cols)
            prev_duals = duals

        # column management
        new_cols = []
        for i in data['I']:
            col, rc = solve_pricer_for_site(i, duals, data, prev_duals=prev_duals)
            if col is not None and rc > 1e-6:
                # Check if similar column already exists
                similar_exists = False
                if i in column_pool:
                    for existing_col in column_pool[i]:
                        if (all(existing_col.x[c] == col.x[c] for c in data['C']) and
                            existing_col.pv == col.pv and existing_col.batt == col.batt):
                            similar_exists = True
                            break
                
                if not similar_exists:
                    new_cols.append(col)
                    # Add to column pool
                    if i not in column_pool:
                        column_pool[i] = []
                    column_pool[i].append(col)

        for col in new_cols: col.last_used = outer  # Track when column was last used
        logging.info(f"[Outer {outer} | CG {it}] Added {len(new_cols)} columns")
        
        # Remove old columns with low usage
        if len(cols) > 1500:  # Arbitrary threshold
            # Remove columns with low lambda values
            cols = [col for col in cols if hasattr(col, 'last_used') and 
                    (outer - col.last_used) <= 3 or
                    sum(col.x.values()) + col.pv + col.batt > 0]

        logging.info(f"[Outer {outer}] Solving integer RMP with {len(cols)} columns...")

        # integer solve on restricted master
        mip, _, _ = solve_RMP(cols, data, integrality=True)
        if mip is None or mip.status != GRB.OPTIMAL: 
            break

        if mip is not None and mip.status == GRB.OPTIMAL:
            logging.info(f"[Outer {outer}] Integer RMP obj = {mip.objVal:,.2f} | MIPGap={getattr(mip, 'MIPGap', 0.0):.4%}")

        # check pricing at the integer incumbent (relax, get duals, price again)
        rlp, duals, lam_vals = solve_RMP(cols, data, integrality=False,
                                         slack_penalty_override=1e6,
                                         prev_lambda_vals=prev_lambda_vals,
                                         prox_weight=5e-3)
        prev_lambda_vals = lam_vals
        improv = False
        for i in data['I']:
            col, rc = solve_pricer_for_site(i, duals, data, prev_duals=prev_duals)
            if col is not None and rc > 1e-6:
                cols.append(col)
                improv = True
        if not improv:
            return mip, cols  # nothing more to add- accept
        prev_duals = duals

        # convergence checking - use current_obj_val instead of obj_val
        if abs(current_obj_val - best_obj) < 1e-4:
            convergence_count += 1
        else:
            convergence_count = 0
            
        #convergence criteria
        if (convergence_count >= 2 or  # Small objective change
            (best_obj > -1e100 and current_obj_val < best_obj) or  # Objective worsening
            (outer > 0 and len(new_cols) == 0)):  # No new columns
            logging.info(f"Convergence criteria met at outer iteration {outer}")
            break
            
        best_obj = current_obj_val        
    # fallback final integer
    mip, _, _ = solve_RMP(cols, data, integrality=True)
    return mip, cols

In [5]:
def solve_RMP(columns, data, integrality=False, slack_penalty_override=None, prev_lambda_vals=None, prox_weight=1e-3):
    I, M, H, C = data['I'], data['MONTHS'], data['H'], data['C']
    N_MONTH, DAYS = data['N_MONTH'], data['DAYS']
    pv_cf, PV_slot_at_cf1 = data['pv_cf'], data['PV_slot_at_cf1']
    T, x_kWh, redir_min = data['T'], data['x_kWh'], data['redir_min']
    K, Price, tou = data['K'], data['Price'], data['tou']
    CL, pv_upper = data['CL'], data['pv_upper']
    BATT_MAX, BATT_CELL_CAP = data['BATT_MAX'], data['BATT_CELL_CAP']
    eta_ch, eta_dis = data['eta_ch'], data['eta_dis']
    prev_month, prev_dict = data['prev_month'], data['prev_dict']
    M_CH = data['M_CH']
    penalty_slack = data['penalty_slack'] if slack_penalty_override is None else slack_penalty_override
    SLOT_HOURS = 0.5  # 30-min slot

    m = gp.Model("RMP_full")
    m.Params.OutputFlag = 0
    if integrality:
        m.Params.MIPGap = 1e-4
    else:
        m.Params.Method = 1        # primal simplex to retrieve duals reliably
    m.Params.DualReductions = 0    # keep rows in dual form (avoid vanishing Pi)
    m.Params.NumericFocus = 1      # Improve numerical stability
    m.Params.FeasibilityTol = 1e-6  # Tighter feasibility tolerance
    m.Params.OptimalityTol = 1e-6  # Tighter optimality tolerance

    vtype_lambda = GRB.BINARY if integrality else GRB.CONTINUOUS
    vtype_trip   = GRB.INTEGER if integrality else GRB.CONTINUOUS
    vtype_yarc   = GRB.BINARY  if integrality else GRB.CONTINUOUS

    lam = [m.addVar(vtype=vtype_lambda, lb=0, name=f"lam_{k}")
           for k in range(len(columns))]

    edisp = {(i,mon,t,c,b): m.addVar(lb=0, name=f"ed_{i}_{mon}_{t}_{c}_{b}")
             for i in I for mon in M for t in H for c in C for b in ['home','public']}
    grid_dir  = {(i,mon,t): m.addVar(lb=0, name=f"gd_{i}_{mon}_{t}") for i in I for mon in M for t in H}
    grid_batt = {(i,mon,t): m.addVar(lb=0, name=f"gb_{i}_{mon}_{t}") for i in I for mon in M for t in H}
    pv_dir    = {(i,mon,t): m.addVar(lb=0, name=f"pd_{i}_{mon}_{t}") for i in I for mon in M for t in H}
    pv_batt   = {(i,mon,t): m.addVar(lb=0, name=f"pb_{i}_{mon}_{t}") for i in I for mon in M for t in H}
    batt_dis  = {(i,mon,t): m.addVar(lb=0, name=f"bd_{i}_{mon}_{t}") for i in I for mon in M for t in H}
    soc       = {(i,mon,t): m.addVar(lb=0, name=f"soc_{i}_{mon}_{t}") for i in I for mon in M for t in H}
    delta     = {(i,mon,t): m.addVar(vtype=vtype_yarc, lb=0, ub=1, name=f"delta_{i}_{mon}_{t}")
                 for i in I for mon in M for t in H}
    slack     = {(i,mon,t,b): m.addVar(lb=0, name=f"s_{i}_{mon}_{t}_{b}")
                 for i in I for mon in M for t in H for b in ['home','public']}

    A = data['A']
    z     = {(i,j,mon,t): m.addVar(lb=0, name=f"z_{i}_{j}_{mon}_{t}") for (i,j,mon,t) in A}
    Yarc  = {(i,j,mon,t): m.addVar(vtype=vtype_yarc, lb=0, ub=1, name=f"Y_{i}_{j}_{mon}_{t}") for (i,j,mon,t) in A}
    ntrip = {(i,j,mon,t): m.addVar(vtype=vtype_trip, lb=0, name=f"n_{i}_{j}_{mon}_{t}") for (i,j,mon,t) in A}
    conv_rows  = {}
    limit_rows = {}
    pv_con     = {}
    pvUB_rows  = {}
    battUB_rows= {}
    bUp_rows   = {}
    bLo_rows   = {}
    bJan_rows  = {}

    # Convexity (one per site)
    for i in I:
        conv_rows[i] = m.addConstr(quicksum(lam[k]
                                           for k in range(len(columns))
                                           if columns[k].i == i) == 1,
                                   name=f"conv_{i}")

    #  Per-type capacity limits (kWh per slot) 
    cap_type = {}
    for i in I:
        for c in C:
            x_count = quicksum(lam[k]*columns[k].x.get(c, 0) for k in range(len(columns)) if columns[k].i == i)
            for mon in M:
                for t in H:
                    cap_type[(i,mon,t,c)] = m.addConstr(
                        quicksum(edisp[i,mon,t,c,b] for b in ['home','public'])
                        <= K[c] * x_count,
                        name=f"cap_{i}_{mon}_{t}_{c}"
                    )
    
    # Demand coverage
    dem = data['Demand']
    for i in I:
        for mon in M:
            for t in H:
                m.addConstr(quicksum(edisp[i,mon,t,c,'home'] for c in C) + slack[(i,mon,t,'home')]
                            == dem[(i,mon,t,'home')], name=f"demH_{i}_{mon}_{t}")
                outflow = quicksum(z[i,j,mon,t] for j in I if (i,j,mon,t) in A)
                inflow  = quicksum(z[j,i,mon,t] for j in I if (j,i,mon,t) in A)
                m.addConstr(quicksum(edisp[i,mon,t,c,'public'] for c in C) + slack[(i,mon,t,'public')]
                            == dem[(i,mon,t,'public')] - outflow + inflow, name=f"demP_{i}_{mon}_{t}")

    # Redirection linkages (linearized)
    zcap_rc_rows = {}   # duals for z <= rec_cap, used in pricing for "incoming" value
    
    for (i,j,mon,t) in A:
        # Recipient capacity as a linear expression of lam (no product with Yarc)
        rec_cap = quicksum(
                        lam[k] * sum(columns[k].x.get(c,0) * K[c] for c in C)
                        for k in range(len(columns)) if columns[k].i == j
                    )

    # Use McCormick envelopes for better linearization
        # z <= rec_cap * Yarc (implemented as zcapRC)
        # z >= redir_min * Yarc (implemented as zmin)
        # z <= x_kWh * ntrip (implemented as ztrip)
        
        #  minimum energy if the arc is active
        m.addConstr(z[i,j,mon,t] >= redir_min * Yarc[(i,j,mon,t)], name=f"zmin_{i}_{j}_{mon}_{t}")
        # capacity limit driven by the chosen design at j  (meaningful dual)
        zcap_rc_rows[(i,j,mon,t)] = m.addConstr(z[i,j,mon,t] <= rec_cap, name=f"zcapRC_{i}_{j}_{mon}_{t}")
        # on/off link by a static big-M (no bilinear term)
        m.addConstr(z[i,j,mon,t] <= data['M_redir'][j] * Yarc[(i,j,mon,t)], name=f"zcapY_{i}_{j}_{mon}_{t}")
        # energy-per-trip link
        m.addConstr(z[i,j,mon,t] <= x_kWh * ntrip[(i,j,mon,t)], name=f"ztrip_{i}_{j}_{mon}_{t}")

        # Yarc <= ntrip (each trip requires at least one binary activation)
        m.addConstr(Yarc[(i,j,mon,t)] <= ntrip[(i,j,mon,t)], name=f"Yarc_ntrip_{i}_{j}_{mon}_{t}")
        
        # ntrip <= M_big * Yarc (M_big should be carefully chosen)
        M_big = math.ceil(data['M_redir'][j] / x_kWh)
        m.addConstr(ntrip[(i,j,mon,t)] <= M_big * Yarc[(i,j,mon,t)], name=f"ntrip_Yarc_{i}_{j}_{mon}_{t}")

    # Charger count bound (per site)
    for i in I:
        total_chars = quicksum(lam[k]*sum(columns[k].x.get(c,0) for c in C)
                               for k in range(len(columns)) if columns[k].i == i)
        limit_rows[i] = m.addConstr(total_chars <= CL[i], name=f"limit_{i}")

    # PV generation envelope
    for i in I:
        for mon in M:
            for t in H:
                pv_count = quicksum(lam[k]*columns[k].pv for k in range(len(columns)) if columns[k].i == i)
                pv_con[(i,mon,t)] = m.addConstr(
                    pv_dir[i,mon,t] + pv_batt[i,mon,t] <= PV_slot_at_cf1 * pv_cf[mon][t] * pv_count,
                    name=f"pv_{i}_{mon}_{t}"
                )

    # PV & battery installation bounds
    for i in I:
        pv_count  = quicksum(lam[k]*columns[k].pv   for k in range(len(columns)) if columns[k].i == i)
        batt_ct   = quicksum(lam[k]*columns[k].batt for k in range(len(columns)) if columns[k].i == i)
        pvUB_rows[i]   = m.addConstr(pv_count  <= pv_upper[i], name=f"pvUB_{i}")
        battUB_rows[i] = m.addConstr(batt_ct   <= BATT_MAX,   name=f"battUB_{i}")

    # SOC bounds and dynamics for Pi
    LAST_H = max(H)
    for i in I:
        for mon in M:
            for t in H:
                batt_ct = quicksum(lam[k]*columns[k].batt for k in range(len(columns)) if columns[k].i == i)
                bUp_rows[(i,mon,t)] = m.addConstr(soc[i,mon,t] <= 0.9 * BATT_CELL_CAP * batt_ct, name=f"socUp_{i}_{mon}_{t}")
                bLo_rows[(i,mon,t)] = m.addConstr(soc[i,mon,t] >= 0.1 * BATT_CELL_CAP * batt_ct, name=f"socLo_{i}_{mon}_{t}")
    for i in I:
        batt_ct = quicksum(lam[k]*columns[k].batt for k in range(len(columns)) if columns[k].i == i)
        bJan_rows[i] = m.addConstr(soc[i,'January',1] == 0.20 * BATT_CELL_CAP * batt_ct, name=f"socJan_{i}")

    for i in I:
        for mon in M:
            for t in H:
                if mon == 'January' and t == 1:
                    continue
                if t == 1:
                    pm = prev_month[mon]
                    m.addConstr(soc[i,mon,t] == soc[i,pm,LAST_H], name=f"socCarry_{i}_{mon}_{t}")
                else:
                    prev_t = prev_dict[t]
                    m.addConstr(
                        soc[i,mon,t] == soc[i,mon,prev_t] + eta_ch*(grid_batt[i,mon,t] + pv_batt[i,mon,t]) - (1.0/eta_dis)*batt_dis[i,mon,t],
                        name=f"socDyn_{i}_{mon}_{t}"
                    )

    bCh_rows = {}
    bDis_rows = {}
    # battery rate + exclusivity
    for i in I:
        batt_ct = quicksum(lam[k]*columns[k].batt for k in range(len(columns)) if columns[k].i == i)
        for mon in M:
            for t in H:
                bCh_rows[(i,mon,t)] = m.addConstr(
                    grid_batt[i,mon,t] + pv_batt[i,mon,t] <= M_CH * batt_ct * delta[i,mon,t],
                    name=f"battChargeRate_{i}_{mon}_{t}"
                )
                bDis_rows[(i,mon,t)] = m.addConstr(
                    batt_dis[i,mon,t] <= M_CH * batt_ct * (1 - delta[i,mon,t]),
                    name=f"battDisRate_{i}_{mon}_{t}"
                )
                

    # Annual SOC closure: Dec,last == Jan,1
    for i in I: m.addConstr(soc[i,'December', max(H)] == soc[i,'January',1], name=f"socClose_{i}")

    for i in I:
        for mon in M:
            for t in H:
                m.addConstr(
                    grid_dir[i,mon,t] + pv_dir[i,mon,t] + batt_dis[i,mon,t]
                    == quicksum(edisp[i,mon,t,c,b] for c in C for b in ['home','public']),
                    name=f"energyBal_{i}_{mon}_{t}"
                )

    # Prox only for LP solves, and only if we have previous lambda values
    if (not integrality) and (prev_lambda_vals is not None) and len(prev_lambda_vals) == len(lam):
        prox = quicksum(prox_weight * (lam[k] - prev_lambda_vals[k]) * (lam[k] - prev_lambda_vals[k])
                        for k in range(len(columns)))
    else:
        prox = 0.0

    revenue    = quicksum(N_MONTH[mon]*Price[c]*edisp[i,mon,t,c,b] for i in I for mon in M for t in H for c in C for b in ['home','public'])
    # tiny nudge to break degeneracy and produce nonzero CAPTYPE duals
    eps_push = 1e-4
    nudge = eps_push * quicksum(N_MONTH[mon]*edisp[i,mon,t,c,b] for i in I for mon in M for t in H for c in C for b in ['home','public'])
    
    grid_cost  = quicksum(N_MONTH[mon]*tou[mon][t]*(grid_dir[i,mon,t] + grid_batt[i,mon,t]) for i in I for mon in M for t in H)
    incentive  = quicksum(N_MONTH[mon]*T[(i,j)]*ntrip[(i,j,mon,t)] for (i,j,mon,t) in A)
    capex      = DAYS * quicksum(lam[k]*columns[k].fixed_cost for k in range(len(columns)))
    slack_pen  = penalty_slack * quicksum(N_MONTH[mon]*slack[(i,mon,t,b)] for i in I for mon in M for t in H for b in ['home','public'])
    m.setObjective(revenue - grid_cost - incentive - capex - slack_pen + nudge - prox, GRB.MAXIMIZE)

    m.optimize()

    # status
    if m.status not in [GRB.OPTIMAL, GRB.SUBOPTIMAL]:
        logging.warning(f"RMP solve status: {m.status}")
        # Try to recover by solving from scratch
        m.reset()
        m.optimize()

    # - duals -
    duals = {}
    if m.status == GRB.OPTIMAL and not integrality:
        for key,row in cap_type.items():
            try: duals[('CAPTYPE',) + key] = row.Pi  # key=(i,mon,t,c)
            except: duals[('CAPTYPE',) + key] = 0.0
        for key,row in pv_con.items():
            try: duals[('PVGEN',)+key] = row.Pi
            except: duals[('PVGEN',)+key] = 0.0
        for key,row in bUp_rows.items():
            try: duals[('BUP',)+key] = row.Pi
            except: duals[('BUP',)+key] = 0.0
        for key,row in bLo_rows.items():
            try: duals[('BLO',)+key] = row.Pi
            except: duals[('BLO',)+key] = 0.0
        for i,row in pvUB_rows.items():
            try: duals[('PVUB', i)] = row.Pi
            except: duals[('PVUB', i)] = 0.0
        for i,row in battUB_rows.items():
            try: duals[('BCAP', i)] = row.Pi
            except: duals[('BCAP', i)] = 0.0
        for i,row in bJan_rows.items():
            try: duals[('BJAN', i)] = row.Pi
            except: duals[('BJAN', i)] = 0.0
        for i,row in conv_rows.items():
            try: duals[('CONV', i)] = row.Pi
            except: duals[('CONV', i)] = 0.0
        for i,row in limit_rows.items():
            try: duals[('CL', i)] = row.Pi
            except: duals[('CL', i)] = 0.0
        for key,row in zcap_rc_rows.items():
            try: duals[('ZCAPRC',)+key] = row.Pi  # key = (i,j,mon,t)
            except: duals[('ZCAPRC',)+key] = 0.0
        for key,row in bCh_rows.items():
            try: duals[('BCHR',)+key] = row.Pi
            except: duals[('BCHR',)+key] = 0.0
        for key,row in bDis_rows.items():
            try: duals[('BDIS',)+key] = row.Pi
            except: duals[('BDIS',)+key] = 0.0
    lam_vals = None
    if (m.status == GRB.OPTIMAL) and (not integrality):
        lam_vals = [lam[k].X for k in range(len(columns))]
    return m, duals, lam_vals

In [6]:
def solve_pricer_for_site(i, duals, data, prev_duals=None, omega=0.75, stabilization="simple"):
    """
    Build the best design (x_slow, x_med, x_fast, PV, Batt) for site i
    against the current duals. Uses:
      - α_cap(i,mon,t) from site-capacity rows
      - φ_pv(i,mon,t) from PV-generation rows
      - β_up/β_lo/β_jan from battery SOC rows
      - μ_pvUB(i), μ_bcap(i), μ_CL(i) from bounds
      - π_i from convexity row (CONV)
    Reduced cost (annual units) of a column z for site i:

      rc(z) =  [ Σ_{mon,t} α_cap(i,mon,t) * ( Σ_c K_c x_c ) ]
             + [ Σ_{mon,t} φ_pv(i,mon,t) * ( PV_slot_at_cf1 * cf(mon,t) * PV ) ]
             + [ BATT_CELL_CAP * (0.9 Σβ_up - 0.1 Σβ_lo + 0.20 β_jan) * Batt ]
             + μ_pvUB(i) * PV + μ_bcap(i) * Batt + μ_CL(i) * (Σ_c x_c)
             - DAYS * ( Σ_c daily_cost[c] * x_c + daily_cost['PV']*PV + daily_cost['Batt']*Batt )
             - π_i

    If rc>0 --> add column.
    """
    C, K = data['C'], data['K']
    M, H = data['MONTHS'], data['H']
    pv_cf, PV_slot_at_cf1 = data['pv_cf'], data['PV_slot_at_cf1']
    CL, pv_upper = data['CL'], data['pv_upper']
    BATT_MAX, BATT_CELL_CAP = data['BATT_MAX'], data['BATT_CELL_CAP']
    DAYS = data['DAYS']; dc = data['daily_cost']
    SLOT_HOURS = 0.5
    
    M_CH = data['M_CH']  # Battery charge/discharge rate limit
    eta_ch = data['eta_ch']  # Charging efficiency
    eta_dis = data['eta_dis']  # Discharging efficiency
    prev_month = data['prev_month']  # Previous month mapping
    prev_dict = data['prev_dict']  # Previous time slot mapping
    LAST_H = max(H)  # Last time slot

    # simple dual smoothing (helps early degeneracy)
    def sm(key, method=stabilization):
        if prev_duals is None:
            return duals.get(key, 0.0)
        
        if method == "simple":
            return (1-omega)*prev_duals.get(key, 0.0) + omega*duals.get(key, 0.0)
        elif method == "weighted":
            # Weight recent duals more heavily
            recent_weight = 0.7
            return (1-recent_weight)*prev_duals.get(key, 0.0) + recent_weight*duals.get(key, 0.0)
        elif method == "adaptive":
            # Adaptive weighting based on dual change
            prev_val = prev_duals.get(key, 0.0)
            curr_val = duals.get(key, 0.0)
            change = abs(curr_val - prev_val)
            weight = max(0.1, min(0.9, 1.0 - change / (1.0 + abs(prev_val))))
            return (1-weight)*prev_val + weight*curr_val
        else:
            return duals.get(key, 0.0)
    
    # per-type capacity value
    alpha_cap = {c: sum(sm(('CAPTYPE', i, mon, t, c), "adaptive") for mon in M for t in H) for c in C}
    alpha_pv = sum(sm(('PVGEN', i, mon, t), "adaptive") * PV_slot_at_cf1 * pv_cf[mon][t] for mon in M for t in H)

    # battery value from SOC bounds + rate
    beta_up = sum(sm(('BUP', i, mon, t)) for mon in M for t in H)
    beta_lo = sum(sm(('BLO', i, mon, t)) for mon in M for t in H)
    beta_jan = sm(('BJAN', i))
    alpha_batt  = BATT_CELL_CAP*(0.9*beta_up - 0.1*beta_lo + 0.20*beta_jan)
    alpha_rate  = sum(sm(('BCHR', i, mon, t)) + sm(('BDIS', i, mon, t)) for mon in M for t in H)

    # bounds + convexity
    mu_pvUB = sm(('PVUB', i))
    mu_bcap = sm(('BCAP', i))
    mu_CL   = sm(('CL',   i))
    pi_i    = sm(('CONV', i))

    m = gp.Model(f"Price_{i}")
    m.Params.OutputFlag = 0
    m.Params.MIPGap = 1e-4
    m.Params.TimeLimit = 30  # Increased time limit for more complex pricing
    m.Params.Method = 1           # primal simplex (yields duals)
    m.Params.DualReductions = 0   # keep rows in the dual

    x = {c: m.addVar(vtype=GRB.INTEGER, lb=0, ub=CL[i], name=f"x_{c}") for c in C}
    pv = m.addVar(vtype=GRB.INTEGER, lb=0, ub=pv_upper[i], name="pv")
    batt = m.addVar(vtype=GRB.INTEGER, lb=0, ub=BATT_MAX, name="batt")
    m.addConstr(quicksum(x[c] for c in C) <= CL[i], name="limit")

    cap_sum = quicksum(K[c]*x[c] for c in C)

    # incoming redirection value: z <= rec_cap row at (k -> i)
    alpha_in = sum(sm(('ZCAPRC', k, i, mon, t)) for mon in M for t in H for k in data['I'] if (k, i, mon, t) in data['A'])

    # Main reduced cost CAPACITY PART: multiply by SLOT_HOURS
    rc_cap = quicksum(SLOT_HOURS * K[c] * (alpha_cap[c] + alpha_in) * x[c] for c in C)

    # PV part stays (from PVGEN duals): already in kWh, no extra slot factor needed
    rc_pv  = alpha_pv * pv

    # Battery part: SoC bounds + (rate duals)*M_CH
    rc_bat = (alpha_batt + M_CH * alpha_rate) * batt

    # Bounds/convexity duals + daily costs * DAYS
    rc_pen = mu_pvUB*pv + mu_bcap*batt + mu_CL*quicksum(x[c] for c in C)
    rc_cost= DAYS*(quicksum(dc[c]*x[c] for c in C) + dc['PV']*pv + dc['Batt']*batt)

    rc = rc_cap + rc_pv + rc_bat + rc_pen - rc_cost - pi_i
    
    m.setObjective(rc, GRB.MAXIMIZE)
    m.optimize()

    # Debug: show best reduced cost for site i
    try:
        logging.debug(f"[Pricer i={i}] best RC = {m.ObjVal:,.2f} | x={ {c:int(round(x[c].X)) for c in C} } pv={int(round(pv.X))} batt={int(round(batt.X))}")
    except Exception: pass
    if m.status != GRB.OPTIMAL: return None, -1e9
    if m.ObjVal <= 1e-6: return None, m.ObjVal

    xval  = {c:int(round(x[c].X)) for c in C}
    pvval = int(round(pv.X)); battval = int(round(batt.X))
    cap_total = sum(K[c]*xval[c] for c in C)
    fixed = sum(dc[c]*xval[c] for c in C) + dc['PV']*pvval + dc['Batt']*battval
    return Column(i, xval, pvval, battval, cap_total, fixed), m.ObjVal

def column_generation(data, max_iter=30, tol=1e-4):
    cols = initial_columns(data); best = -1e100
    for it in range(max_iter):
        rmp, duals, _ = solve_RMP(cols, data, integrality=False, slack_penalty_override=1e6)
        if rmp.status != GRB.OPTIMAL: break
        obj = rmp.objVal
        logging.info(f"Iter {it}: RMP = {obj:.2f}")
        if obj <= best + tol and it>0: break
        best = obj
        new_cols=[]
        for i in data['I']:
            col, rc = solve_pricer_for_site(i, duals, data)
            if col is not None and rc > 1e-6:
                new_cols.append(col)
        if not new_cols:
            logging.info("No positive-reduced-cost columns.")
            break
        cols.extend(new_cols)
        logging.info(f"Added {len(new_cols)} columns.")
    return cols

In [7]:
def solve_final_integer(cols, data):
    m, _, _ = solve_RMP(cols, data, integrality=True)
    if m.status != GRB.OPTIMAL:
        return None, None
    return m, cols

def summarize_solution(model, cols, data):
    I, M, H, C = data['I'], data['MONTHS'], data['H'], data['C']
    N_MONTH, DAYS = data['N_MONTH'], data['DAYS']
    Price, tou = data['Price'], data['tou']
    A = data['A']; T = data['T']

    lam = [model.getVarByName(f"lam_{k}") for k in range(len(cols))]
    installed = {i:{c:0 for c in C} for i in I}
    pv_inst = {i:0 for i in I}
    batt_inst = {i:0 for i in I}
    for k,col in enumerate(cols):
        val = lam[k].X if lam[k] is not None else 0.0
        if val>0.5:
            for c in C: installed[col.i][c] += col.x[c]
            pv_inst[col.i] += col.pv
            batt_inst[col.i] += col.batt

    tot_by_type = {c:0 for c in C}
    for i in I:
        for c in C: tot_by_type[c] += installed[i][c]

    revenue=grid=inc=capex=slack=0.0
    for i in I:
        for m in M:
            for t in H:
                e_sum = 0.0
                for c in C:
                    for b in ['home','public']:
                        e = model.getVarByName(f"ed_{i}_{m}_{t}_{c}_{b}").X
                        revenue += N_MONTH[m]*Price[c]*e
                        e_sum += e
                grid += N_MONTH[m]*tou[m][t]*(
                    model.getVarByName(f"gd_{i}_{m}_{t}").X +
                    model.getVarByName(f"gb_{i}_{m}_{t}").X
                )
                slack += data['penalty_slack']*N_MONTH[m]*(
                    model.getVarByName(f"s_{i}_{m}_{t}_home").X +
                    model.getVarByName(f"s_{i}_{m}_{t}_public").X
                )
    for (i,j,m,t) in A:
        inc += N_MONTH[m]*T[(i,j)]*model.getVarByName(f"n_{i}_{j}_{m}_{t}").X
    for k,col in enumerate(cols):
        lamk = lam[k].X if lam[k] is not None else 0.0
        capex += DAYS*lamk*col.fixed_cost

    redir_energy = 0.0; trips = 0.0
    for (i,j,m,t) in A:
        z = model.getVarByName(f"z_{i}_{j}_{m}_{t}")
        n = model.getVarByName(f"n_{i}_{j}_{m}_{t}")
        if z is not None: redir_energy += N_MONTH[m]*z.X
        if n is not None: trips += N_MONTH[m]*n.X

    pv_dir_annual = 0.0; pv_batt_annual=0.0; grid_dir_annual=0.0; grid_batt_annual=0.0; batt_dis_annual=0.0
    for i in I:
        for m in M:
            for t in H:
                pv_dir_annual  += N_MONTH[m]*model.getVarByName(f"pd_{i}_{m}_{t}").X
                pv_batt_annual += N_MONTH[m]*model.getVarByName(f"pb_{i}_{m}_{t}").X
                grid_dir_annual+= N_MONTH[m]*model.getVarByName(f"gd_{i}_{m}_{t}").X
                grid_batt_annual+=N_MONTH[m]*model.getVarByName(f"gb_{i}_{m}_{t}").X
                batt_dis_annual+=N_MONTH[m]*model.getVarByName(f"bd_{i}_{m}_{t}").X

    print("\n DW (full: chargers + PV + battery + redirection) ")
    print(f"Objective (annual profit): {model.objVal:.2f} SEK/yr")
    try:
        print(f"MIP gap: {model.MIPGap:.4%}")
    except Exception:
        pass

    print(" Installed public chargers ")
    for c in C: print(f"  {c:6s}: {int(round(tot_by_type[c]))} units")
    print(f"PV panels      : {int(round(sum(pv_inst.values())))} units")
    print(f"Battery cells  : {int(round(sum(batt_inst.values())))} units")
    print(" Annual cost components (SEK/yr) ")
    print(f"  Revenue from charging: {revenue:.2f}")
    print(f"  Grid purchase cost:   {grid:.2f}")
    print(f"  Redirection incentive:{inc:.2f}")
    print(f"  CapEx (annuitized):   {capex:.2f}")
    print(f"  Slack penalty:        {slack:.2f}")
    print(" Annual energy flows (kWh) ")
    print(f"  Grid purchased (direct): {grid_dir_annual:.1f}")
    print(f"  Grid → battery charging: {grid_batt_annual:.1f}")
    print(f"  PV used directly:        {pv_dir_annual:.1f}")
    print(f"  PV → battery charging:   {pv_batt_annual:.1f}")
    print(f"  Battery discharge:       {batt_dis_annual:.1f}")
    print(f"  Energy redirected:       {redir_energy:.1f}")
    print(f"  Total trips:             {int(round(trips))}")

In [1]:
def main():
    data = load_data()
    daily_home  = sum(data['Demand'][(i, data['MONTHS'][0], t, 'home')]   for i in data['I'] for t in data['H'])
    daily_pub   = sum(data['Demand'][(i, data['MONTHS'][0], t, 'public')] for i in data['I'] for t in data['H'])
    annual_home = sum(data['N_MONTH'][m]*sum(data['Demand'][(i,m,t,'home')]   for i in data['I'] for t in data['H']) for m in data['MONTHS'])
    annual_pub  = sum(data['N_MONTH'][m]*sum(data['Demand'][(i,m,t,'public')] for i in data['I'] for t in data['H']) for m in data['MONTHS'])
    logging.info(f"Daily kWh: home={daily_home:.1f}, public={daily_pub:.1f}")
    logging.info(f"Annual kWh: home={annual_home:.1f}, public={annual_pub:.1f}")

    # stabilized outer loop with repricing 
    mip, cols = solve_with_repricing(data, max_outer=3, max_cg=25)
    if mip is None or mip.status != GRB.OPTIMAL:
        print("No solution.")
        return
    summarize_solution(mip, cols, data)

if __name__ == "__main__":
    start_wall = datetime.now()
    start_cpu  = perf_counter()
    print(f"[START] {start_wall:%Y-%m-%d %H:%M:%S}")
    main()

    end_wall = datetime.now()
    end_cpu  = perf_counter()
    elapsed_s = end_cpu - start_cpu
    elapsed_min = elapsed_s / 60.0
    print(f"[END]   {end_wall:%Y-%m-%d %H:%M:%S}")
    print(f"[ELAPSED] {elapsed_s:,.2f} s  (~{elapsed_min:,.2f} min)")

[START] 2025-09-21 21:46:22


INFO: Daily kWh: home=40.4, public=19356.1
INFO: Annual kWh: home=14735.5, public=7064979.3


Set parameter Username


INFO: Set parameter Username


Set parameter LicenseID to value 2622223


INFO: Set parameter LicenseID to value 2622223


Academic license - for non-commercial use only - expires 2026-02-14


INFO: Academic license - for non-commercial use only - expires 2026-02-14
INFO: [Outer 0 | CG 0] LP-RMP obj = 32,745,020.13  | cols=2016
INFO:   RC max/min/avg: -0.0 / -0.0 / 0.0
INFO: No positive-reduced-cost columns at this LP.
INFO: [Outer 0 | CG 0] Added 0 columns
INFO: [Outer 0] Solving integer RMP with 1950 columns...
INFO: [Outer 0] Integer RMP obj = 31,291,567.39 | MIPGap=0.0099%



================== DW (full: chargers + PV + battery + redirection) — Final ==================
Objective (annual profit): 31291567.39 SEK/yr
MIP gap: 0.0099%
----- Installed public chargers -----
  slow  : 1 units
  medium: 215 units
  fast  : 26 units
PV panels      : 4356 units
Battery cells  : 116 units
----- Annual cost components (SEK/yr) -----
  Revenue from charging: 43647382.98
  Grid purchase cost:   9044219.54
  Redirection incentive:14346.49
  CapEx (annuitized):   3297957.54
  Slack penalty:        0.00
----- Annual energy flows (kWh) -----
  Grid purchased (direct): 4971950.2
  Grid → battery charging: 198529.3
  PV used directly:        1533326.8
  PV → battery charging:   435142.2
  Battery discharge:       574437.8
  Energy redirected:       5566.1
  Total trips:             456
[END]   2025-09-22 05:31:11
[ELAPSED] 27,889.71 s  (~464.83 min)


DW fails to acquire optimal solution (high gap) due to multiple degeneracy for the small nudge in intial stages...limiting scope for improvement.
High computational requirement as the convergence iterates through every feasible column. **Try Benders approach!**